In [332]:
import csv
import string
import pandas as pd

In [333]:
train = pd.read_csv('SentimentStemmed.csv', comment='#')
train = train.drop([train.columns[0] , train.columns[1]] ,  axis=1)


In [334]:
#train = pd.read_csv('https://raw.githubusercontent.com/twishaavyas/CyberPolice/master/Kaggle/train.csv', comment='#')

In [335]:
train.head()

,Insult,Comment,Sentiment
0,1,you fuck your dad,-0.400000
1,0,i realli dont understand your pointxa0 it seem...,0.000000
2,0,axc2xa0major of canadian can and ha been wrong...,0.283333
3,0,listen if you dont wanna get marri to a man or...,0.450000
4,0,cxe1c bu1ea1n xuu1ed1ng u0111u01b0u1eddng biu1...,0.000000


In [336]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit(train['Comment'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [337]:
X = tfidf.transform(train['Comment'])
train['Comment'][0]

'you fuck your dad'

In [338]:
print([X[0, tfidf.vocabulary_['fuck']]])

[0.436196640782194]


In [339]:
from sklearn.model_selection import train_test_split

X = train.drop('Insult', axis=1)
y = train['Insult']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [340]:
print("Train set has total {0} entries with {1:.2f}% not insults, {2:.2f}% insults".format(len(X_train),
                                                                             (len(X_train[y_train == 0]) / (len(X_train)*1.))*100,
                                                                            (len(X_train[y_train == 1]) / (len(X_train)*1.))*100))

Train set has total 2960 entries with 73.65% not insults, 26.35% insults


In [342]:
import numpy as np
from sklearn import tree
from sklearn.preprocessing import StandardScaler

vect_word = TfidfVectorizer(max_features=20000, analyzer='word',dtype=np.float32)
tr_vect = vect_word.fit_transform(train['Comment'])
temp = train.drop(['Insult', 'Comment'] , axis = 1)
X = np.concatenate((tr_vect.toarray(), train['Sentiment'].reshape((3947, 1))), 1)
y = train.Insult
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [343]:
from sklearn.metrics import accuracy_score
print ("Accuracy is ", accuracy_score(y_test, pred)*100)

Accuracy is  77.10233029381965


In [344]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

In [345]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
logreg.fit(X_train, y_train)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)

In [346]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.10f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.8156028369


In [347]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(
   confusion_matrix(y_test, y_pred),
   columns=['Predicted Not Bully', 'Predicted Bully'],
   index=['True Not Bully', 'Bully']
)

,Predicted Not Bully,Predicted Bully
True Not Bully,646,72
Bully,110,159


In [348]:
from sklearn.svm import SVC  
svclassifier = SVC(kernel='rbf')  
svclassifier.fit(X_train, y_train)  
y_pred = svclassifier.predict(X_test)  

In [349]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print ("Accuracy is ", accuracy_score(y_test, y_pred)*100)

[[718   0]
 [269   0]]
             precision    recall  f1-score   support

          0       0.73      1.00      0.84       718
          1       0.00      0.00      0.00       269

avg / total       0.53      0.73      0.61       987

Accuracy is  72.74569402228977


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [350]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
print(principalDf.head())

X_train, X_test, y_train, y_test = train_test_split(principalDf, y, random_state = 0)
svclassifier = SVC(kernel='linear')  
svclassifier.fit(X_train, y_train)  
y_pred = svclassifier.predict(X_test)  
print ("Accuracy is ", accuracy_score(y_test, y_pred)*100)

   principal component 1  principal component 2
0              -0.430729              -0.185917
1              -0.004492              -0.069223
2               0.291942               0.133066
3               0.450328               0.033055
4               0.002615              -0.101174
Accuracy is  72.74569402228977
